In [1]:
import numpy as np
import pandas as pd
import os
import glob
import codecs
import scispacy
import spacy
import sklearn
import warnings 

from collections import defaultdict, Counter
from sklearn.model_selection import KFold
# from negspacy.negation import Negex
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import defaultdict
from transformers import BertTokenizer
from nltk.corpus import stopwords

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:

  
warnings.filterwarnings(action = 'ignore') 
  
# print(model1.similarity("Hospice", "Immunology"))                               
# nlp0 = spacy.load("en_core_web_sm")
# negex = Negex(nlp0, language = "en_clinical")
# nlp0.add_pipe(negex, last=True)
# doc0 = nlp0("he is not an emotional eater")
import string

#string punctuation
punct = set(string.punctuation)
# for e in doc0.ents:
# 	print(e.text, e._.negex)

# Using scispacy with "en_core_sci_sm":
nlp = spacy.load("en_core_sci_sm")

# # Using scispacy with "en_core_sci_lg":
# nlp = spacy.load("en_core_sci_lg")

# # Using scispacy with "en_ner_bc5cdr_md:
# nlp = spacy.load("en_ner_bc5cdr_md")


def tokenize_doc(text):
    bow = defaultdict(float)
    temp = []
    doc = nlp(text)
    doc = doc.ents
    lowered_tokens = map(lambda t: str(t).lower(), doc)

    for token in lowered_tokens:
        # token = token.split(".,")
        # if (len(token) > 1):
        #     for j in token:
        #         j = j.split(":,")
        #         for i in j:
        #             bow[i] += 1.0
        #             temp.append(i)
        # else:
        #     for i in token:
        #         bow[i] += 1.0
        #         temp.append(i)

        temp.append(token)
    return temp

def tokenize_doc2(text):
    t = BertTokenizer.from_pretrained("bert-large-uncased-vocab.txt", lowercase=True)
    warnings.filterwarnings('ignore')
    output = t.tokenize(text)
    temp = []
    words = [w for w in output if w not in punct]
    for token in words:
        temp.append(token)
    return temp

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv("mtsamples.csv")

In [19]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]
data = data.drop(temp)

        

In [20]:
data = data.reset_index(drop=True)

In [24]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]

surgery = list(data[data["medical_specialty"] == ' Surgery'].index.values)
consult = list(data[data["medical_specialty"] == ' Consult - History and Phy.'].index.values)
cardiovascular = list(data[data["medical_specialty"] == ' Cardiovascular / Pulmonary'].index.values)
orthopedic = list(data[data["medical_specialty"] == ' Orthopedic'].index.values)
radiology = list(data[data["medical_specialty"] == ' Radiology'].index.values)

general = list(data[data["medical_specialty"] == ' General Medicine'].index.values)
gastroenterology = list(data[data["medical_specialty"] == ' Gastroenterology'].index.values)
neurology = list(data[data["medical_specialty"] == ' Neurology'].index.values)
soap = list(data[data["medical_specialty"] == ' SOAP / Chart / Progress Notes'].index.values)
obstetrics = list(data[data["medical_specialty"] == ' Obstetrics / Gynecology'].index.values)

urology = list(data[data["medical_specialty"] == ' Urology'].index.values)
discharge = list(data[data["medical_specialty"] == ' Discharge Summary'].index.values)
ent = list(data[data["medical_specialty"] == ' ENT - Otolaryngology'].index.values)
neurosurgery = list(data[data["medical_specialty"] == ' Neurosurgery'].index.values)
hematology = list(data[data["medical_specialty"] == ' Hematology - Oncology'].index.values)

ophthalmology = list(data[data["medical_specialty"] == ' Ophthalmology'].index.values)
Nephrology = list(data[data["medical_specialty"] == ' Nephrology'].index.values)


uniq_dict = {}

temp_list = surgery[76:] + consult[76:] + cardiovascular[76:] + orthopedic[76:] + radiology[76:] + general[76:] + gastroenterology[76:] + neurology[76:] + soap[76:] + obstetrics[76:] + urology[76:] + discharge[76:] + ent[76:] + neurosurgery[76:] + hematology[76:] + ophthalmology[76:] + Nephrology[76:]

In [13]:

test = tokenize_doc2(data['transcription'][0])   
print(test) 

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


['subjective', 'this', '23', 'year', 'old', 'white', 'female', 'presents', 'with', 'complaint', 'of', 'all', '##er', '##gies', 'she', 'used', 'to', 'have', 'all', '##er', '##gies', 'when', 'she', 'lived', 'in', 'seattle', 'but', 'she', 'thinks', 'they', 'are', 'worse', 'here', 'in', 'the', 'past', 'she', 'has', 'tried', 'cl', '##ari', '##tin', 'and', 'z', '##yr', '##tec', 'both', 'worked', 'for', 'short', 'time', 'but', 'then', 'seemed', 'to', 'lose', 'effectiveness', 'she', 'has', 'used', 'all', '##eg', '##ra', 'also', 'she', 'used', 'that', 'last', 'summer', 'and', 'she', 'began', 'using', 'it', 'again', 'two', 'weeks', 'ago', 'it', 'does', 'not', 'appear', 'to', 'be', 'working', 'very', 'well', 'she', 'has', 'used', 'over', 'the', 'counter', 'spray', '##s', 'but', 'no', 'prescription', 'nasal', 'spray', '##s', 'she', 'does', 'have', 'asthma', 'but', 'does', '##t', 'not', 'require', 'daily', 'medication', 'for', 'this', 'and', 'does', 'not', 'think', 'it', 'is', 'fl', '##aring', 'up'

In [14]:
print(tokenize_doc(data['transcription'][0])  )

['subjective', 'white female', 'complaint', 'allergies', 'allergies', 'lived', 'seattle', 'thinks', 'claritin', 'zyrtec', 'short time', 'lose', 'effectiveness', 'allegra', 'summer', 'weeks', 'working', 'prescription', 'nasal sprays', 'asthma', 'doest', 'daily', 'medication', 'flaring', 'up.,medications', 'medication', 'tri-cyclen', 'medicine', 'allergies.,objective:,vitals', 'weight', 'pounds', 'blood pressure 124/78.,heent', 'throat', 'erythematous', 'exudate', 'nasal mucosa', 'erythematous', 'swollen', 'drainage', 'tms', 'clear.,neck', 'supple', 'adenopathy.,lungs', 'clear.,assessment', 'allergic rhinitis.,plan:,1', 'zyrtec', 'allegra again', 'loratadine', 'prescription', 'coverage', 'cheaper.,2', 'samples', 'nasonex', 'nostril', 'weeks', 'prescription']


In [15]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer


kf = KFold(n_splits=12, random_state=None, shuffle=False)
kf.get_n_splits(data)

Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_doc2)
features  = Tfidf_vect.fit(data['transcription'])


In [16]:
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

nsvc = NuSVC(nu=0.0001, probability=True)
lsvc = LinearSVC(tol = 1e-5, C = 0.0001, random_state=500000)

In [17]:
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data):
    # print("TRAIN:", len(train_index), "TEST:", len(test_index))
    # for train_index, test_index in kf.split(data):
    x_train = []
    y_train = []

    x_train, x_test = data['transcription'][train_index], data['transcription'][test_index]
    y_train, y_test = data['medical_specialty'][train_index], data['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
#     nsvc.fit(Train_X_Tfidf,y_train) 
    lsvc.fit(Train_X_Tfidf,y_train) 
    
#     predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())


    print(accuracy_score(y_test, predictions_nuSVM)*100)
#     total += accuracy_score(y_test, predictions_nuSVM)*100
#     count += 1

# print(count)
# print(total/count)
print(total/count)
print(res)
    


                             precision    recall  f1-score   support

       Allergy / Immunology    0.00000   0.00000   0.00000         0
                 Bariatrics    0.00000   0.00000   0.00000         0
 Cardiovascular / Pulmonary    0.00000   0.00000   0.00000         0
                  Dentistry    0.00000   0.00000   0.00000         0
           General Medicine    0.00000   0.00000   0.00000         0
                  Neurology    0.00000   0.00000   0.00000         0
                    Surgery    1.00000   0.57488   0.73006       414
                    Urology    0.00000   0.00000   0.00000         0

                   accuracy                        0.57488       414
                  macro avg    0.12500   0.07186   0.09126       414
               weighted avg    1.00000   0.57488   0.73006       414

57.48792270531401
              precision    recall  f1-score   support

     Surgery    1.00000   1.00000   1.00000       414

    accuracy                        1.000

In [18]:
x_train, x_test = data['transcription'][0:5], data['transcription'][0:5]
y_train, y_test = data['medical_specialty'][0:5], data['medical_specialty'][0:5]
                                                  
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)

nsvc.fit(Train_X_Tfidf,y_train)   
#classifier2.fit(Train_X_Tfidf,y_train)

predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
print(accuracy_score(predictions_nuSVM, y_test)*100)

100.0


In [ ]:
print(len(Tfidf_vect.vocabulary_))

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
print(Tfidf_vect.get_feature_names())